# Pascal segmentation

The Pascal VOC challenge is a very popular dataset for building and evaluating algorithms for image classification, object detection, and segmentation.

Classes:
`background`, `aeroplane`, `bicycle`, `bird`, `boat`, `bottle`, `bus`, `car`, `cat`, `chair`, `cow`, `diningtable`, `dog`, `horse`, `motorbike`, `person`, `pottedplant`, `sheep`, `sofa`, `train`, `tvmonitor`

255 is the ignore label that marks pixels excluded from learning and
evaluation by the PASCAL VOC ground truth.

#### 1. Necessary imports

In [ ]:
import sys

import PIL
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

sys.path.append('..')
from batchflow import B, V, F, R, P, W
from batchflow.opensets import PascalSegmentation
from batchflow.models.torch import TorchModel, UNet, ResUNet
from batchflow.models.metrics import ClassificationMetrics

plt.style.use('seaborn-poster')
plt.style.use('ggplot')

In [ ]:
def draw_data(label, prediction):
    _, ax = plt.subplots(1, 2, figsize=(15, 15))
    ax[0].set_title('Mask')
    ax[0].imshow(label)
    ax[0].grid()
    ax[1].set_title('Prediction')
    ax[1].imshow(prediction)
    ax[1].grid()
    plt.show()

#### 2. Load data

In [ ]:
ds = PascalSegmentation(bar=True)

In [ ]:
# We should construct batch and get images.
batch = ds.train.p.next_batch(16)
images = batch.images
labels = batch.labels

#### 3. Define training pipeline

In [ ]:
NUM_CLASSES = 22
model_config = {
    'inputs/targets/classes': NUM_CLASSES,
    'body/encoder/num_stages': 4,
    'body/decoder/blocks/filters': [512, 256, 128, 64],
    'body/encoder/blocks/filters': [64, 128, 256, 512],
    'body/embedding/filters': 512,
    'head': dict(layout='c', filters=NUM_CLASSES, kernel_size=1),

    'optimizer': ('Adam', {'lr': 0.001}),
}

In [ ]:
# Define constants
BATCH_SIZE = 50
N_EPOCHS = 200
SIZE = (160, 160)

In [ ]:
def process_mask(x):
    x = np.squeeze(x)
    np.place(x, x==255, 21)
    return x

In [ ]:
# Define actions for training
train_ppl = (ds.train.p
    .init_model('dynamic', UNet, 'model', config=model_config)
    .init_variable('loss', [])
    .resize(size=SIZE, src='images', dst='images')
    .resize(size=SIZE, src='labels', dst='labels')
    .to_array(channels='first', src='images', dst='images')
    .to_array(channels='first', src='labels', dst='labels')
    .apply_transform_all(src='labels', dst='labels', func=process_mask)
    .train_model('model', B('images'), B('labels'), fetches='loss', save_to=V('loss', mode='a'))
    .run_later(BATCH_SIZE, n_epochs=N_EPOCHS, drop_last=True, shuffle=42, bar='n')
)

In [ ]:
# Here we will run it
train_ppl.run()

In [ ]:
plt.plot(train_ppl.v('loss')[:])

#### 4. ... and test pipeline

In [ ]:
# Define actions for test
test_ppl = (ds.test.p
    .import_model('model', train_ppl)
    .resize(size=SIZE, src='images', dst='images')
    .resize(size=SIZE, src='labels', dst='labels')
    .to_array(channels='first', src='images', dst='images')
    .to_array(channels='first', src='labels', dst='labels')
    .apply_transform_all(src='labels', dst='labels', func=process_mask)
    .predict_model('model', B('images'), fetches='predictions', save_to=B('predictions'))
)

#### 4.1 Calculate metrics for one batch

In [ ]:
batch = test_ppl.next_batch(50)

In [ ]:
labels = batch.labels
predictions = np.argmax(batch.predictions, axis=1)

draw_data(labels[4], predictions[4])

In [ ]:
# we will use f1 score form other library

from sklearn.metrics import f1_score
f1_score_wh_zeros = f1_score(labels.ravel(), predictions.ravel().astype(int),
                             labels=np.arange(NUM_CLASSES), average=None)

print("F1 score for each class:\n", f1_score_wh_zeros)

print('\n\nResulted f1: ', np.mean(f1_score_wh_zeros[f1_score_wh_zeros != 0]))

#### 4.2. Predictions for all test data

In [ ]:
# Define actions for test without gather_metrics
test_ppl = (ds.test.p
    .import_model('model', train_ppl)
    .init_variable('targets', [])
    .init_variable('predictions', [])
    .resize(size=SIZE, src='images', dst='images')
    .resize(size=SIZE, src='labels', dst='labels')
    .to_array(channels='first', src='images', dst='images')
    .to_array(channels='first', src='labels', dst='labels')
    .apply_transform_all(src='labels', dst='labels', func=process_mask)
    .predict_model('model', B('images'), fetches='predictions', save_to=V('predictions', mode='a'))
    .update(V('targets', mode='a'), B('labels'))
)

In [ ]:
test_ppl.run(4, drop_last=False, bar='n')

In [ ]:
print(type(test_ppl.v('targets')))

In [ ]:
# getting a labels and predictions from pipeline
labels = np.array(test_ppl.v('targets'))
print('labels.shape: ', labels.shape)

print('labels.shape after concatenation: ', np.concatenate(labels).shape)

labels = np.concatenate(np.concatenate(labels))

predictions = np.array(test_ppl.v('predictions'))
predictions = np.concatenate(predictions)
print('predictions shape after concatenation: ', predictions.shape)

predictions = np.argmax(predictions, axis=1)
print('predictions shape after argmax: ', predictions.shape)

In [ ]:
f1_score_wh_zeros = f1_score(labels.ravel(), predictions.ravel().astype(int),
                             labels=np.arange(NUM_CLASSES), average=None)

print("F1 score for each class:\n", f1_score_wh_zeros)

print('\n\nResulted f1: ', np.mean(f1_score_wh_zeros[f1_score_wh_zeros != 0]))

#### 5. Save model

In [ ]:
#save model from pipeline

In [ ]:
train_ppl.save_model_now('model', 'my_model')

In [ ]:
#get model from pipeline

In [ ]:
my_model = train_ppl.m('model')

In [ ]:
#this is still batchflow model
print(my_model)

In [ ]:
#save torch model from batchflow model

In [ ]:
my_model.save('my_model_2')

In [ ]:
#how to get pytorch model from batchflow
pure_torch = my_model.model

#### 6. Load model

In [ ]:
import os
load_config = {
    'device': 'gpu:0',
    'load/path': './my_model'
}

In [ ]:
load_ppl = (ds.test.p
    #here we are gonna load the model
    .init_model('dynamic', UNet, 'model', config=load_config)
    .init_variable('metrics', None)
    .init_variable('targets', [])
    .init_variable('preds', [])
    .resize(size=SIZE, src='images', dst='images')
    .resize(size=SIZE, src='labels', dst='labels')
    .to_array(channels='first', src='images', dst='images')
    .to_array(channels='first', src='labels', dst='labels')
    .update(V('targets', mode='a'), B('labels'))
    .apply_transform_all(src='labels', dst='labels', func=process_mask)
    .predict_model('model', B('images'), fetches='predictions', save_to=B('predictions'))
    .update(V('preds',mode='a'), B('predictions')))

In [ ]:
#check whether the model loaded or not
batch = load_ppl.next_batch(10)

In [ ]:
labels = batch.labels
predictions = np.argmax(batch.predictions, axis=1)

draw_data(labels[4], predictions[4])

#### Task:

Try to modify the model and training process to get better f1 score.

#### What you can change?
For example, you can add additional keys into dictionary and then try to vary them:

```
model_config = {
                'inputs/targets/classes': NUM_CLASSES,
                'body/encoder/num_stages': 4,
                'body/decoder/blocks/filters': [128, 64, 64, 32],
                'body/encoder/blocks/filters': [32, 64, 64, 128],
                'body/embedding/filters': 128,
                'head': dict(layout='c', filters=NUM_CLASSES, kernel_size=1),
    
                'optimizer': ('Adam', {'lr': 0.001}),
            }
```